In [25]:
PATH = '/home/g056122/FeatExt_Data_Clustering/final_github/content_based_filtering'

In [24]:
DATAPATH = '/home/g056122/ISG_studienarbeit/Movielens_100K'

In [1]:
# import required libraries
import numpy as np
import pandas as pd
import datetime
from kmodes.kprototypes import KPrototypes
import matplotlib.pyplot as plt
import pickle

In [2]:
# import movies data
dfmovies = pd.read_csv("/home/g056122/ISG_studienarbeit/Movielens_100K/u.item", sep='|', header=None, engine='python', encoding='latin-1')
dfmovies.columns = ['movieId', 'movietitle', 'releasedate', 'videoreleasedate','imdburl', 'unknown', 'action', 'adventure', 'animation','childrens', 'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'film-Noir', 'horror', 'musical', 'mystery', 'romance', 'sci-fi','thriller', 'war', 'western']

In [3]:
dfmovies = dfmovies.drop(['videoreleasedate', 'imdburl'], axis=1)

In [4]:
dfmovies['releasedate'] = pd.to_datetime (dfmovies['releasedate'], format='%d-%b-%Y')
dfmovies['year']=dfmovies['releasedate'].dt.year

In [5]:
dfmovies.head()

,movieId,movietitle,releasedate,unknown,action,adventure,animation,childrens,comedy,crime,...,film-Noir,horror,musical,mystery,romance,sci-fi,thriller,war,western,year
0,1,Toy Story (1995),1995-01-01,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,1995.0
1,2,GoldenEye (1995),1995-01-01,0,1,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1995.0
2,3,Four Rooms (1995),1995-01-01,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1995.0
3,4,Get Shorty (1995),1995-01-01,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1995.0
4,5,Copycat (1995),1995-01-01,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,1995.0


In [6]:
dfmovies = dfmovies.drop(['movietitle', 'releasedate'], axis=1)

In [7]:
n_dfmovies = pd.get_dummies(dfmovies, columns = ['year'])

In [8]:
n_dfmovies.head()

,movieId,unknown,action,adventure,animation,childrens,comedy,crime,documentary,drama,...,year_1989.0,year_1990.0,year_1991.0,year_1992.0,year_1993.0,year_1994.0,year_1995.0,year_1996.0,year_1997.0,year_1998.0
0,1,0,0,0,1,1,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,2,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,4,0,1,0,0,0,1,0,0,1,...,0,0,0,0,0,0,1,0,0,0
4,5,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,1,0,0,0


In [9]:
n_dfmovies.describe()

,movieId,unknown,action,adventure,animation,childrens,comedy,crime,documentary,drama,...,year_1989.0,year_1990.0,year_1991.0,year_1992.0,year_1993.0,year_1994.0,year_1995.0,year_1996.0,year_1997.0,year_1998.0
count,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,...,1682.000000,1682.000000,1682.00000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000
mean,841.500000,0.001189,0.149227,0.080262,0.024970,0.072533,0.300238,0.064804,0.029727,0.431034,...,0.008918,0.014269,0.01308,0.021998,0.074911,0.127229,0.130202,0.211058,0.170036,0.038644
std,485.695893,0.034473,0.356418,0.271779,0.156081,0.259445,0.458498,0.246253,0.169882,0.495368,...,0.094041,0.118632,0.11365,0.146719,0.263326,0.333329,0.336626,0.408181,0.375776,0.192803
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,421.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,841.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1261.750000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1682.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
categoryindex = list(range(1,n_dfmovies.shape[1]))

In [11]:
num_clusters = 8
kprototype = KPrototypes(n_clusters=num_clusters, init = "Huang")
kprototype.fit_predict(n_dfmovies, categorical = categoryindex)    

array([1, 1, 1, ..., 0, 0, 0], dtype=uint16)

In [13]:
# cluster centroids
centroids = kprototype.cluster_centroids_

In [14]:
centroids.dtype, centroids.shape

(dtype('float64'), (8, 91))

In [15]:
n_dfmovies = n_dfmovies.values

In [16]:
n_dfmovies.shape

(1682, 91)

In [17]:
# euclidean_distances
euc_dis = []
for i in range(n_dfmovies.shape[0]):
    for k in range(num_clusters):
        dist = 0
        for j in range(n_dfmovies.shape[1]):
            val = (n_dfmovies[i][j] - centroids[k][j])**2
            dist = dist + val
        ed = np.sqrt(dist)
        euc_dis.append(ed)
euc_dis = np.array(euc_dis)
euc_dis = euc_dis.reshape((n_dfmovies.shape[0],num_clusters))

In [18]:
euc_dis.shape

(1682, 8)

In [19]:
val_m = np.array(dfmovies.movieId.values)
val_m = val_m.reshape(euc_dis.shape[0],1)
val_m.shape

(1682, 1)

In [20]:
# merge dataset
n_dfmovies = np.concatenate((val_m, n_dfmovies), axis=1)
n_dfmovies, n_dfmovies.shape

(array([[   1,    1,    0, ...,    0,    0,    0],
        [   2,    2,    0, ...,    0,    0,    0],
        [   3,    3,    0, ...,    0,    0,    0],
        ...,
        [1680, 1680,    0, ...,    0,    0,    1],
        [1681, 1681,    0, ...,    0,    0,    0],
        [1682, 1682,    0, ...,    1,    0,    0]]),
 (1682, 92))

In [21]:
n_dfmovies = np.concatenate((n_dfmovies, euc_dis), axis=1)
n_dfmovies.shape

(1682, 100)

In [22]:
n_dfmovies

array([[1.00000000e+00, 1.00000000e+00, 0.00000000e+00, ...,
        9.47002112e+02, 5.29503777e+02, 1.36400183e+03],
       [2.00000000e+00, 2.00000000e+00, 0.00000000e+00, ...,
        9.46002114e+02, 5.28503784e+02, 1.36300183e+03],
       [3.00000000e+00, 3.00000000e+00, 0.00000000e+00, ...,
        9.45001058e+02, 5.27501896e+02, 1.36200110e+03],
       ...,
       [1.68000000e+03, 1.68000000e+03, 0.00000000e+00, ...,
        7.32002049e+02, 1.14950130e+03, 3.15003175e+02],
       [1.68100000e+03, 1.68100000e+03, 0.00000000e+00, ...,
        7.33001364e+02, 1.15050087e+03, 3.16004747e+02],
       [1.68200000e+03, 1.68200000e+03, 0.00000000e+00, ...,
        7.34001362e+02, 1.15150087e+03, 3.17001577e+02]])

In [26]:
# import ratings dataset
dfratings = pd.read_csv(f"{DATAPATH}/u.data", sep='\t', header=None, engine='python', encoding='latin-1')
dfratings.columns = ['userId', 'movieId', 'rating', 'timestamp']

In [27]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
scaler = MinMaxScaler()
dfratings['timestamp'] = pd.to_datetime (dfratings['timestamp'], unit='s')
dfratings['year']=dfratings['timestamp'].dt.year
dfratings[['year', 'rating']]=scaler.fit_transform(dfratings[['year', 'rating']])
dfratings = dfratings.drop('timestamp', axis=1)

In [28]:
n_dfratings = dfratings.drop('rating', axis=1).values
n_ratings = dfratings['rating'].values

In [29]:
n_dfratings

array([[1.96e+02, 2.42e+02, 0.00e+00],
       [1.86e+02, 3.02e+02, 1.00e+00],
       [2.20e+01, 3.77e+02, 0.00e+00],
       ...,
       [2.76e+02, 1.09e+03, 0.00e+00],
       [1.30e+01, 2.25e+02, 0.00e+00],
       [1.20e+01, 2.03e+02, 0.00e+00]])

In [30]:
# merge dataset
list =[]
for m in range(n_dfratings.shape[0]):
    for n in range(n_dfmovies.shape[0]):
        if (n_dfratings[m][1] == n_dfmovies[n][0]):
            merge = np.concatenate((n_dfmovies[n], n_dfratings[m]))
            list.append(merge)
list = np.array(list)
new_list = []
# delete duplicate movieId
for k in range(list.shape[0]):
    val = np.delete(list[k], 0)
    new_list.append(val)
new_list = np.array(new_list)

In [31]:
new_list.shape

(100000, 102)

In [32]:
# split X, y
X = new_list
y = n_ratings

In [33]:
# Kfold
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True, random_state=32)

In [34]:
from sklearn.metrics import mean_squared_error,mean_absolute_error 
from sklearn.ensemble import RandomForestRegressor

In [35]:
classes = []
truth_val = []
for i, (train_index, test_index) in enumerate(kfold.split(X, y)):
  val = i+1
  print(f'Fold {val}:')
  # define model 
  rf_model = RandomForestRegressor(n_estimators = 100)
  # model fit
  rf_model.fit(X[train_index],y[train_index])
  # model predict
  predict = rf_model.predict(X[test_index])
  # append predictions and real values
  x =  (predict * 4 ) + 1
  classes.append(x)
  truth = y[test_index]
  truth_val.append(truth)
# convert list to array
classes = np.array(classes)
truth_val = np.array(truth_val)

Fold 1:
Fold 2:
Fold 3:
Fold 4:
Fold 5:


In [36]:
# lists with rmse and mae of 5 folds
rmse, mae = [], []
for i in range(5):
  # root mean square error
  rmse_val = mean_squared_error(truth_val[i], classes[i], squared = False)
  # mean absolute error
  mae_val = mean_absolute_error(truth_val[i], classes[i])
  # append rmse and and mae
  rmse.append(rmse_val)
  mae.append(mae_val)

In [37]:
# results
print(f'RMSE {np.mean(rmse)}, {np.std(rmse)}')
print(f'MAE {np.mean(mae)}, {np.std(mae)}')

RMSE 2.96521368338234, 0.002855675425708581
MAE 2.8912940999999996, 0.0029127063257390366
